In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.4.1


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [3]:
# 간단한 Sequential 모델을 정의합니다
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# 모델 객체를 만듭니다
model = create_model()

# 모델 구조를 출력합니다
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 새로운 콜백으로 모델 훈련하기
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백을 훈련에 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Epoch 1/10
32/32 [==============================] - 1s 34ms/step - loss: 1.5495 - accuracy: 0.5340 - val_loss: 0.6785 - val_accuracy: 0.7990

Epoch 00001: saving model to training_1\cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 5ms/step - loss: 0.4158 - accuracy: 0.9009 - val_loss: 0.5165 - val_accuracy: 0.8420

Epoch 00002: saving model to training_1\cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 5ms/step - loss: 0.2864 - accuracy: 0.9394 - val_loss: 0.4878 - val_accuracy: 0.8500

Epoch 00003: saving model to training_1\cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 5ms/step - loss: 0.1926 - accuracy: 0.9635 - val_loss: 0.4205 - val_accuracy: 0.8790

Epoch 00004: saving model to training_1\cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 5ms/step - loss: 0.1375 - accuracy: 0.9690 - val_loss: 0.4400 - val_accuracy: 0.8570

Epoch 00005: saving model to training_1\cp.ckpt
Epoch 6/10
32/32 [==============================] - 0s

In [5]:
ls {checkpoint_dir}

 C 드라이브의 볼륨: Windows10
 볼륨 일련 번호: 22E4-B69A

 C:\mds\jupyter\training_1 디렉터리

2021-04-21  오후 10:34    <DIR>          .
2021-04-21  오후 10:34    <DIR>          ..
2021-04-21  오후 10:34                71 checkpoint
2021-04-21  오후 10:34         4,886,673 cp.ckpt.data-00000-of-00001
2021-04-21  오후 10:34             1,222 cp.ckpt.index
               3개 파일           4,887,966 바이트
               2개 디렉터리  58,565,914,624 바이트 남음


In [6]:
# 기본 모델 객체를 만듭니다
model = create_model()

# 모델을 평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3072 - accuracy: 0.1140
훈련되지 않은 모델의 정확도: 11.40%


In [7]:
# 가중치 로드
model.load_weights(checkpoint_path)

# 모델 재평가
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4083 - accuracy: 0.8750
복원된 모델의 정확도: 87.50%


In [8]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# 새로운 모델 객체를 만듭니다
model = create_model()

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련합니다
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2\cp-0005.ckpt

Epoch 00010: saving model to training_2\cp-0010.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00020: saving model to training_2\cp-0020.ckpt

Epoch 00025: saving model to training_2\cp-0025.ckpt

Epoch 00030: saving model to training_2\cp-0030.ckpt

Epoch 00035: saving model to training_2\cp-0035.ckpt

Epoch 00040: saving model to training_2\cp-0040.ckpt

Epoch 00045: saving model to training_2\cp-0045.ckpt

Epoch 00050: saving model to training_2\cp-0050.ckpt


In [9]:
ls {checkpoint_dir}

 C 드라이브의 볼륨: Windows10
 볼륨 일련 번호: 22E4-B69A

 C:\mds\jupyter\training_2 디렉터리

2021-04-21  오후 10:37    <DIR>          .
2021-04-21  오후 10:37    <DIR>          ..
2021-04-21  오후 10:37                81 checkpoint
2021-04-21  오후 10:37         1,628,726 cp-0000.ckpt.data-00000-of-00001
2021-04-21  오후 10:37               402 cp-0000.ckpt.index
2021-04-21  오후 10:37         4,886,685 cp-0005.ckpt.data-00000-of-00001
2021-04-21  오후 10:37             1,222 cp-0005.ckpt.index
2021-04-21  오후 10:37         4,886,685 cp-0010.ckpt.data-00000-of-00001
2021-04-21  오후 10:37             1,222 cp-0010.ckpt.index
2021-04-21  오후 10:37         4,886,685 cp-0015.ckpt.data-00000-of-00001
2021-04-21  오후 10:37             1,222 cp-0015.ckpt.index
2021-04-21  오후 10:37         4,886,685 cp-0020.ckpt.data-00000-of-00001
2021-04-21  오후 10:37             1,222 cp-0020.ckpt.index
2021-04-21  오후 10:37         4,886,685 cp-0025.ckpt.data-00000-of-00001
2021-04-21  오후 10:37             1,222 cp-0025.ckpt.index
2021-04-2

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [12]:
# 새로운 모델 객체를 만듭니다
model = create_model()

# 이전에 저장한 가중치를 로드합니다
model.load_weights(latest)

# 모델을 재평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4848 - accuracy: 0.8740
복원된 모델의 정확도: 87.40%


In [13]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 새로운 모델 객체를 만듭니다
model = create_model()

# 가중치를 복원합니다
model.load_weights('./checkpoints/my_checkpoint')

# 모델을 평가합니다
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4848 - accuracy: 0.8740
복원된 모델의 정확도: 87.40%


In [14]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 1.6605 - accuracy: 0.4979
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4783 - accuracy: 0.8684
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2997 - accuracy: 0.9102
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2023 - accuracy: 0.9578
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1474 - accuracy: 0.9708
INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [28]:
# my_model 디렉토리
ls saved_model

# assests 폴더, saved_model.pb, variables 폴더
ls saved_model/my_model

'ls'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'ls'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [29]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [30]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4120 - accuracy: 0.8700
복원된 모델의 정확도: 87.00%
(1000, 10)


In [31]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 3ms/step - loss: 1.6155 - accuracy: 0.5092
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4720 - accuracy: 0.8813
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2832 - accuracy: 0.9308
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2371 - accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1597 - accuracy: 0.9720


In [32]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model('my_model.h5')

# 모델 구조를 출력합니다
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [33]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4320 - accuracy: 0.8680
복원된 모델의 정확도: 86.80%
